In [1]:
import os

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [ ]:
import langchain
langchain.verbose = True

In [2]:
# Add OpenAI library
import openai

In [3]:
# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = "2024-02-01"  # Latest / target version of the API

## Initial tests without a Cloud Vector Store

In [5]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader

In [6]:
SOURCE_FILE_NAME = "/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt"

In [7]:
loader = TextLoader(SOURCE_FILE_NAME)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

Created a chunk of size 180, which is longer than the specified 150
Created a chunk of size 292, which is longer than the specified 150
Created a chunk of size 690, which is longer than the specified 150
Created a chunk of size 534, which is longer than the specified 150
Created a chunk of size 747, which is longer than the specified 150
Created a chunk of size 782, which is longer than the specified 150
Created a chunk of size 645, which is longer than the specified 150
Created a chunk of size 643, which is longer than the specified 150
Created a chunk of size 644, which is longer than the specified 150
Created a chunk of size 726, which is longer than the specified 150
Created a chunk of size 261, which is longer than the specified 150
Created a chunk of size 309, which is longer than the specified 150
Created a chunk of size 1478, which is longer than the specified 150
Created a chunk of size 302, which is longer than the specified 150
Created a chunk of size 447, which is longer th

In [8]:
docs

[Document(page_content='Bible Text: [GEN:1:1-25].\n\nLesson 1 - Senior\n\nMemory Verse: [GEN:1:1-25].', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_content='Cross References"Through faith we understand that the worlds were framed by the word of God, so that things which are seen were not made of things which do appear" \xa0(Hebrews 11:3).', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_content='I The Work of Creation on the First and Second Days\n\n1. God is the Creator of Heaven and earth (the universe), [GEN:1:1].', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_content='2. In the first creative act the earth was without form, without light, and void -- all a shapeless mass, [GEN:1:2].', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_co

In [9]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [37]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = "2024-02-01", openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

In [38]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    openai_embeddings
)

In [22]:
query = "What are the steps to creation?"
docs = db.similarity_search(query)

In [24]:
docs

[Document(page_content='Steps of Creation', metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Senior/Lesson1_Senior.txt'}),
 Document(page_content='The steps of creation day by day are most wonderful. God\'s great purpose was to provide this world for man. In order that man should be sustained here in life, it was necessary that the earth be prepared for the creating of life. The successive creative acts reveal great wisdom and order. God first brought into being the earth, which was without form and void, and darkness was upon the face of the deep. His next act was to command light; for light and the heat, which attends it are essential to human life. He then created the firmament -- the expanse above the earth, modifying the temperature and separating the vaporous waters above (the clouds) from the condensed waters below (the seas). Thus the seas, the atmosphere, and the clouds made perfect provision for life upon the land. Grass, fruits, and herbs were then the next cr

In [14]:
retriever = db.as_retriever()

In [43]:
llm = ChatOpenAI(temperature = 0.0, openai_api_key = os.getenv("API_KEY"), openai_api_base = os.getenv("ENDPOINT"), model_name="gpt-35-turbo", engine="gpt-35-turbo")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [44]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
    )

In [45]:
qa_stuff.get_prompts()

[]

In [48]:
query = "What are the steps to creation? Answer in bullet form"
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Steps of Creation

The steps of creation day by day are most wonderful. God's great purpose was to provide this world for man. In order that man should be sustained here in life, it was necessary that the earth be prepared for the creating of life. The successive creative acts reveal great wisdom and order. God first brought into being the earth, which was without form and void, and darkness was upon the face of the deep. His next act was to command light; for light and the heat, which attends it are essential to human life. He then created the firmament -- the expanse above the earth, modifying the temperature and separating the vaporous waters above (the clouds

In [49]:
print(response)

- God created the earth, which was without form and void, and darkness was upon the face of the deep.
- God commanded light to exist.
- God created the firmament, which is the expanse above the earth, modifying the temperature and separating the vaporous waters above (the clouds) from the condensed waters below (the seas).
- God created grass, fruits, and herbs, each with seed after its kind.
- God created the sun to rule by day, and the moon to rule by night, besides the planets of our solar system, and the innumerable stars, which sparkle like diamonds in the heavens.
- On the seventh day, God rested from all His work.
